In [ ]:
import requests
import urllib.request
import os
from bs4 import BeautifulSoup

### Funkcia na úpravu mena zamestnanca

In [5]:
# funkcia dostane meno zamestnanca s diakritikou a všetkými titulmi
# a vráti len meno bez diakritiky
def modifName(name):
    # rozdelenie mena na jednotlive casti
    name = name.split(" ")
    s = ""
    # odstranenie titulov (heuristika, ze tituly koncia bodkou sa ukazala ako postacujuca)
    for i in name:
        if i[-1] != ".":
            s += i.strip(",")+" "
    # odstranenie diakritiky
    s = s.strip()
    s = s.replace("á", "a")
    s = s.replace("é", "e")
    s = s.replace("í", "i")
    s = s.replace("ó", "o")
    s = s.replace("ú", "u")
    s = s.replace("ý", "y")
    s = s.replace("ž", "z")
    s = s.replace("ť", "t")
    s = s.replace("č", "c")
    s = s.replace("š", "s")
    s = s.replace("ď", "d")
    s = s.replace("ň", "n")
    s = s.replace("ľ", "l")
    s = s.replace("ř", "r")
    s = s.replace("Ž", "Z")
    s = s.replace("Ť", "T")
    s = s.replace("Č", "C")
    s = s.replace("Š", "S")
    s = s.replace("Ď", "D")
    s = s.replace("Ň", "N")
    s = s.replace("Ľ", "L")
    s = s.replace("ô", "o")
    s = s.replace("ä", "a")
    s = s.replace(" ", "_")
    return s
    

### Ziskavanie fotiek

In [6]:
basePhoto = "https://sluzby.fmph.uniba.sk/"
noPhotoFile = open("noPhoto.jpg", "rb")
noPhoto = noPhotoFile.read()
noPhotoFile.close()
  
# funkcia na ziskavanie fotky zamestnanca z jeho podstranky
# dostane meno zamestnanca, url jeho podstranky, skratku jeho katedry a zlozku na ukladanie fotiek
def getImages(href, meno, kat, main):
    # nacitanie podstranky zamestnanca
    personPage = requests.get(href).text
    personParsed = BeautifulSoup(personPage, "html.parser")
    
    # najdenie obrazka na stranke
    temp = personParsed.select("aside.span3 img")
    #print(temp)
    if temp != []:
        for i in temp:
            # vyber z fotografie zamestnanca
            if i["alt"] != "photo":
                # nacitanie obrazok
                img = i["src"]
                imgData = requests.get(basePhoto+img)
                # kontrola ci obrazok nie je placeholder
                if imgData.content != noPhoto:
                    fileName = modifName(meno)
                    # ulozenie obrazka do zlozky na to urcenej s nazvom "[katedra]_[upravene meno].jpg
                    f = open(main+"/"+kat+"_"+fileName+".jpg", "wb")
                    f.write(imgData.content)
                    f.close()
    else:
        # v pripade problemu vypise do konzoly informaciu o ktoreho zamestnanca ide
        print("Pri", meno, "sa vyskytol problem s obrazkami")

# funkcia na ziskavanie fotiek zamestnancou zo stranky katedry
# dotane skratku nazvu katedry, url jej stranky a zlozku na ukladanie fotiek
def findPhotos(baseUrl, kat, main):    
    # nacita stranku katedry
    basePage = requests.get(baseUrl).text
    baseParsed = BeautifulSoup(basePage, "html.parser")

    # nacitanie vsetkych url pre podstranky zamestnancou
    links = baseParsed.select("table a")
    for i, clovek in enumerate(links):
        href = clovek["href"]
        # kontrola ci ide o podstranku zamestnanca
        if "javascript" not in href:
            meno = clovek.text
            # zavolanie funkciu na ziskavanie fotky z podstranky zamestnanca
            getImages(href, meno, kat, main) 
    # oznamenie ukoncenia ziskavania fotografii
    print("Hotovo", kat)

In [ ]:
mainFolder = "photos"

# vytvorenie zlozky na fotografie
try:
    os.mkdir(mainFolder)
except:
    print("Problem, asi uz existujuca zlozka")

# slovnik obsahujuci skratky katedier a url ich stránok
katedry = {"KAG": "https://fmph.uniba.sk/pracoviska/katedra-algebry-a-geometrie/",
           "KAMS": "https://fmph.uniba.sk/pracoviska/katedra-aplikovanej-matematiky-a-statistiky/",
           "KMANM": "https://fmph.uniba.sk/pracoviska/katedra-matematickej-analyzy-a-numerickej-matematiky/",
           "KAFZM": "https://fmph.uniba.sk/pracoviska/katedra-astronomie-fyziky-zeme-a-meteorologie/", 
           "KEF": "https://fmph.uniba.sk/pracoviska/katedra-experimentalnej-fyziky/",
           "KJFB": "https://fmph.uniba.sk/pracoviska/katedra-jadrovej-fyziky-a-biofyziky/",
           "KTF": "https://fmph.uniba.sk/pracoviska/katedra-teoretickej-fyziky/",
           "AIN": "https://fmph.uniba.sk/pracoviska/katedra-aplikovanej-informatiky/",
           "INF": "https://fmph.uniba.sk/pracoviska/katedra-informatiky/",
           "KDMFI": "https://fmph.uniba.sk/pracoviska/katedra-didaktiky-matematiky-fyziky-a-informatiky/",
           "KJP": "https://fmph.uniba.sk/pracoviska/katedra-jazykovej-pripravy/",
           "KTVS": "https://fmph.uniba.sk/pracoviska/katedra-telesnej-vychovy-a-sportu/",
           "HIP_dek": "https://fmph.uniba.sk/pracoviska/dekanat/",
           "HIP_cpp": "https://fmph.uniba.sk/pracoviska/centrum-projektovej-podpory/",
           "HIP_kec": "https://fmph.uniba.sk/pracoviska/kniznicne-a-edicne-centrum/",
           "HIP_vc": "https://fmph.uniba.sk/pracoviska/vypoctove-centrum/",
           "HIP_vl": "https://fmph.uniba.sk/pracoviska/vyvojove-laboratorium/",
           "HIP_sb": "https://fmph.uniba.sk/pracoviska/sprava-budov/"}

# pre kazdu katedru zavola funkciu na ziskavanie fotografii
for k, url in katedry.items():
    findPhotos(url, k, mainFolder)

#### Katedra aplikovanej informatiky naviac

In [ ]:
baseUrl = "https://dai.fmph.uniba.sk"
vsetci = "/w/People/sk"
kat = "AIN_extra"
noPhotoFile2 = open("noPhoto2.jpg", "rb")
noPhoto2 = noPhotoFile2.read()
noPhotoFile2.close()

# upravena verzia na ziskavanie fotografii z katedry aplikovanej informatiky z ich vlastnej stranky
def getImagesExtra(clovek, meno, kat, main):
    # nacitanie stranky s fotografiou
    personPage = requests.get(baseUrl+clovek).text
    personParsed = BeautifulSoup(personPage, "html.parser")
    
    # najdenie obrazkou
    temp = personParsed.select("div#mw-content-text table a.image")
    #print(temp)
    if temp != []:
        for i in temp:
            # vyber fotografie zamestnanca
            if i["title"] != "@":
                # nacitanie stranky s obrazkom
                personPage2 = requests.get(baseUrl+i["href"]).text
                personParsed2 = BeautifulSoup(personPage2, "html.parser")
                
                # finalny vyber obrazka
                temp = personParsed2.select("div.fullImageLink img")
                #print(temp)
                if temp != []:
                    # nacitanie obrazka
                    img = temp[0]["src"]
                    imgData = requests.get(baseUrl+img)
                    # kontrola ci nie je fotografia placeholder
                    if imgData.content != noPhoto2:
                        fileName = modifName(meno)
                        # ulozenie obrazka do zlozky na to urcenej s nazvom "[katedra]_extra_[upravene meno].jpg
                        f = open(main+"/"+kat+"_"+fileName+".jpg", "wb")
                        f.write(imgData.content)
                        f.close()
                else:
                    print("Pri", meno, "sa vyskytol problem s obrazkom 0")
    else:
        print("Pri", meno, "sa vyskytol problem s obrazkom")

# nacitanie stranky so zamestnancami
basePage = requests.get(baseUrl+vsetci).text
baseParsed = BeautifulSoup(basePage, "html.parser")

# ziskavanie podstranok zamestnancov
links = baseParsed.select("table a")
for i, clovek in enumerate(links):
    href = clovek["href"]
    if href[:3] == "/w/":
        # ziskavanie fotografii zamestnancov z ich podstranok
        getImagesExtra(href, clovek["title"], kat, mainFolder)
        
print("Hotovo", kat)        
